# Gauss-Hermite Approximation

### Load packages

In [1]:
from numpy.polynomial.hermite import hermgauss
from matplotlib import pyplot as plt
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy

### Randomized simulation data for two sites
- Set random seed
- Generate 10 true betas ranged in (-10, 10)
- Generate 2 sigmas for noise variance in different sites
- Generate data X1 and X2 with size (1000, 10)
- Generate result y1 and y2 with bernoulli distibution 

In [3]:
np.random.seed(1)
true_beta = (np.random.rand(10,1) - np.random.rand(10,1)) * 10
true_sigma = np.random.rand(2)
X1 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(0, true_sigma[0], 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(0, true_sigma[1], 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)

### Definitions

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0}+\mu_{i0})}{1 + \exp{(X_{ij}^\top\beta_0}+\mu_{i0})}$$

In [4]:
def Pi(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))

Notation for $g(\mu_{i0})$:

$$g(\mu_{i0};\beta_0)=\sum_{j=1}^{n_i}\left[y_{ij}\log\pi_{ij}+(1-y_{ij})\log(1-\pi_{ij})\right]+\log\phi(\mu_{i0};\theta_0)$$

In [32]:
def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

In [33]:
g(X1, y1, mu[0], beta_0)

array([-598.13575178])

Notation for $g'(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \mu_{i0}}=\sum_{j=1}^{n_i}(y_{ij}-\pi_{ij})-\dfrac{\mu_{i0}}{\tau_0^2}$$

In [7]:
def g_1(x, y, mu, beta_0, tau = 1):
    return sum(y - Pi(x, beta_0, mu)) - mu/tau**2

In [34]:
g_1(X1, y1, mu[0], beta_0)

array([-0.46420194])

Notation for $g''(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}^2}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_{i0}}-\dfrac{1}{\tau_0^2}$$

In [8]:
def g_2(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2))) - 1/tau**2

In [35]:
g_2(X1, y1, mu[0], beta_0)

array([-246.67672544])

Notation for $\pi_{ij}'(\beta_0)$:

$$\dfrac{\partial\pi_{ij}}{\partial\beta_0}=\dfrac{X_{ij}\exp\left(X_{ij}^\top\beta_0+\mu_{i0}\right)}{\left[1+\left(X_{ij}^\top\beta_0+\mu_{i0}\right)\right]^2}$$

In [9]:
def Pi_1(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)) * x

Notation for $\hat\omega$:

$$\hat\omega=\sqrt{-\dfrac{1}{g''(\hat\mu_{i0})}}$$

In [10]:
def omega(x, y, mu, beta_0, tau = 1):
    return np.sqrt(-1/g_2(x, y, mu, beta_0))

In [27]:
omega(X1, y1, mu[0], beta_0)

array([0.06367016])

Notation for Hermite polynomial function $f_k(\hat\mu_{i0};\beta_0)$:

$$h_k\exp\{g(\hat\mu_{i0}+\sqrt{2\pi}\hat\omega x_k;\beta_0)+x_k^2\}$$

In [36]:
def f_k(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return h_k * np.exp(g(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0) + x_k**2)

In [37]:
f_k(5, X1, y1, mu[0], beta_0)

array([6.43062083e-266, 1.00836131e-261, 1.61628237e-260, 8.82066164e-262,
       5.14530732e-266])

In [28]:
hermgauss(5)

(array([-2.02018287, -0.95857246,  0.        ,  0.95857246,  2.02018287]),
 array([0.01995324, 0.39361932, 0.94530872, 0.39361932, 0.01995324]))

Notation for $\mathcal L_i$:

$$\mathcal L_i=\sqrt{2\pi}\hat\omega\sum_{k=1}^lh_k\exp\left\{g(\hat\mu_{i0}+\sqrt{2\pi}\hat\omega x_k;\beta_0)+x_k^2\right\}$$

In [12]:
def L(k, x, y, mu, beta_0, tau = 1):
    return np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) * sum(f_k(k, x, y, mu, beta_0))

In [24]:
L(5, X1, y1, mu[0], beta_0)

array([2.88127066e-261])

## STEP 1: Maximize $g(\mu_{i0})$

In [13]:
def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
#         print('Step: ', step, '\n')
        mu_new = mu - g_1(x, y, mu, beta_0, tau)/g_2(x, y, mu, beta_0, tau)
        diff = mu_new - mu
#         print(diff)
        if diff == 0:
#             print(mu)
            break;
        mu = mu_new
    return mu[0]
#     print('mu converges to:', mu[0], 'in', step + 1, 'steps, with difference', diff[0])

## STEP 2: Maximization preparation of $\beta_0$ in LOCAL

Notation for $\dfrac{\partial\mathcal L_i}{\partial\beta_0}$

$$\dfrac{\partial\mathcal L_i}{\partial\beta_0}=\sqrt{2\pi}\hat\omega\sum_{k=1}^l\left\{f_k(\hat\mu_{i0};\beta_0)h_k\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\right\}$$

In [15]:
def L_1(k, x, y, mu, beta_0, tau = 1):
    L_1 = 0
    [x_k, h_k] = hermgauss(k)    
    for i in range(k):
        L_1 += np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) *\
        f_k(k, x, y, mu, beta_0, tau = tau)[i] * h_k[i] *\
        np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                              omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0)
    return L_1

In [23]:
L_1(5, X1, y1, mu[0], beta_0)

array([ 2.56163604e-259, -3.38102716e-259, -7.73779137e-259,
       -2.12535674e-258,  5.04548837e-259, -1.96623897e-258,
       -6.25527959e-259, -6.02953123e-259,  8.45739094e-259,
        1.33023781e-258])

Notation for $\dfrac{\partial^2\mathcal L_i}{\partial\beta_0^2}$:

$$\dfrac{\partial^2\mathcal L_i}{\partial\beta_0^2}=\sqrt{2\pi}\hat\omega\sum_{k=1}^l\left\{f_k(\hat\mu_{i0};\beta_0)h_k^2\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\left[\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\right]^\top+f_k(\hat\mu_{i0};\beta_0)h_k\sum_{j=1}^{n_i}\left(-X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta_0}\right)\right\}$$

In [16]:
def L_2(k, x, y, mu, beta_0, tau = 1):
    L2 = 0
    [x_k, h_k] = hermgauss(k) 
    for i in range(k):
        boogie = 0
        for j in range(x.shape[0]):
            boogie += -x[j].reshape(x[j].shape[0],1) @\
            Pi_1(x[j], beta_0, mu + np.sqrt(2 * np.pi) *\
                 omega(x, y, mu, beta_0, tau) * x_k[i]).reshape(1,x[j].shape[0])
        L2 += np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) *\
        (f_k(k, x, y, mu, beta_0, tau = tau)[i] * h_k[i]**2 *\
         np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                               omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0).reshape(x.shape[1],1) @\
         np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                               omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0).reshape(1,x.shape[1]) +\
        f_k(k, x, y, mu, beta_0, tau = tau)[i] * h_k[i] * boogie)
    return L2

In [22]:
L_2(5, X1, y1, mu[0], beta_0)

array([[ 1.32104425e-257, -3.06369382e-257, -7.09070345e-257,
        -1.95711878e-256,  4.64222762e-257, -1.80608003e-256,
        -5.79365574e-257, -5.50416525e-257,  7.76773780e-257,
         1.22139746e-256],
       [-3.06369382e-257,  3.07722313e-257,  9.43363292e-257,
         2.57885830e-256, -6.10960279e-257,  2.38168873e-256,
         7.65988754e-257,  7.32300013e-257, -1.02774764e-256,
        -1.61358956e-256],
       [-7.09070345e-257,  9.43363292e-257,  2.05042806e-256,
         5.91313990e-256, -1.40173786e-256,  5.47337753e-256,
         1.74122211e-256,  1.67885747e-256, -2.35402240e-256,
        -3.69864341e-256],
       [-1.95711878e-256,  2.57885830e-256,  5.91313990e-256,
         1.61432598e-255, -3.85260620e-256,  1.50271839e-255,
         4.78408080e-256,  4.60782508e-256, -6.46548403e-256,
        -1.01664490e-255],
       [ 4.64222762e-257, -6.10960279e-257, -1.40173786e-256,
        -3.85260620e-256,  8.16026432e-257, -3.56702919e-256,
        -1.13279566e-256

## STEP 3: Maximization of $\beta_0$ in GLOBAL

In [17]:
def LL_1(site_num, k, X, Y, mu, beta_0, tau = 1):
    LL1 = 0
    for i in range(site_num):
        LL1 += L_1(k, X[i], Y[i], mu[i], beta_0, tau) / L(k, X[i], Y[i], mu[i], beta_0, tau)
    return LL1

In [18]:
def LL_2(site_num, k, X, Y, mu, beta_0, tau = 1):
    LL2 = 0
    for i in range(site_num):
        LL2 += L_2(k, X[i], Y[i], mu[i], beta_0, tau) / L(k, X[i], Y[i], mu[i], beta_0, tau)
    LL1 = LL_1(site_num, k, X, Y, mu, beta_0, tau)
    return LL2 - LL1.reshape(LL1.shape[0],1) @ LL1.reshape(1,LL1.shape[0])

In [19]:
def update(site_num, k, X, Y, mu, beta_0, tau = 1):
    direction = LL_1(site_num, k, X, Y, mu, beta_0, tau).reshape(1,beta_0.shape[0]) @\
    inv(LL_2(site_num, k, X, Y, mu, beta_0, tau))
    return direction

# Simulation starts below

### Main function

In [21]:
beta_0 = np.repeat(0, 10).reshape(10,1)
# betea_0 = true_beta
mu = [0.1, 0.1]
X = [X1, X2]
Y = [y1, y2]
k = 2
tau = 1
print('Initial beta:', beta_0, "\n")
for step in range(10):
    print('Step ', step+1, ':\n')
    for i in range(len(mu)): 
        mu[i] = max_mu(X[i], Y[i], mu[i], beta_0)
    for substep in range(100):
        print('Mu:\n', mu, '\n')
        new_beta = beta_0 - update(len(mu), k, X, Y, mu, beta_0).reshape(beta_0.shape[0],1)
        delta = new_beta - beta_0
        beta_0 = new_beta
        print('Beta:\n', beta_0, '\n')
        print('Delta:\n', delta, '\n')

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 2.56858803e-06]
 [ 2.05240478e-05]
 [-1.63266808e-04]
 [-3.94564552e-04]
 [ 8.29763812e-05]
 [-3.98956387e-04]
 [-1.45247191e-04]
 [-1.32584246e-04]
 [ 1.72001376e-04]
 [ 2.36316696e-04]] 

Delta:
 [[ 2.56858803e-06]
 [ 2.05240478e-05]
 [-1.63266808e-04]
 [-3.94564552e-04]
 [ 8.29763812e-05]
 [-3.98956387e-04]
 [-1.45247191e-04]
 [-1.32584246e-04]
 [ 1.72001376e-04]
 [ 2.36316696e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 5.14971153e-06]
 [ 4.10922883e-05]
 [-3.26824099e-04]
 [-7.89823297e-04]
 [ 1.66101590e-04]
 [-7.98615989e-04]
 [-2.90751920e-04]
 [-2.65400625e-04]
 [ 3.44309369e-04]
 [ 4.73045553e-04]] 

Delta:
 [[ 2.58112350e-06]
 [ 2.05682405e-05]
 [-1.63557291e-04]
 [-3.95258745e-04]
 [ 8.31252086e-05]
 [-3.99659602e-04]
 [-1.45504729e-04]
 [-1.32816379e-04]
 [ 1.72307994e-04]
 [ 2.36728857e-04]] 

Mu:
 [0.05579130

Beta:
 [[ 5.09675533e-05]
 [ 3.97508196e-04]
 [-3.15323265e-03]
 [-7.61904293e-03]
 [ 1.60275385e-03]
 [-7.70414039e-03]
 [-2.80537740e-03]
 [-2.56007117e-03]
 [ 3.32205781e-03]
 [ 4.56263024e-03]] 

Delta:
 [[ 2.79745002e-06]
 [ 2.13176969e-05]
 [-1.68732613e-04]
 [-4.07635343e-04]
 [ 8.57751411e-05]
 [-4.12206711e-04]
 [-1.50144509e-04]
 [-1.36965627e-04]
 [ 1.77774907e-04]
 [ 2.44075534e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 5.37779013e-05]
 [ 4.18869605e-04]
 [-3.32228413e-03]
 [-8.02744169e-03]
 [ 1.68869219e-03]
 [-8.11712149e-03]
 [-2.95581090e-03]
 [-2.69729325e-03]
 [ 3.50016976e-03]
 [ 4.80715880e-03]] 

Delta:
 [[ 2.81034797e-06]
 [ 2.13614088e-05]
 [-1.69051479e-04]
 [-4.08398760e-04]
 [ 8.59383413e-05]
 [-4.12981100e-04]
 [-1.50433506e-04]
 [-1.37222077e-04]
 [ 1.78111953e-04]
 [ 2.44528560e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 5.66011648e-05]
 [ 4.40274652e-04]
 [-3.49165613e-03]
 [-8.43660797e-03]
 [ 1.774794

Beta:
 [[ 0.00010659]
 [ 0.00081074]
 [-0.00642167]
 [-0.015514  ]
 [ 0.00326448]
 [-0.01568814]
 [-0.00571522]
 [-0.00521281]
 [ 0.00676589]
 [ 0.00928898]] 

Delta:
 [[ 3.04536377e-06]
 [ 2.21300632e-05]
 [-1.75083383e-04]
 [-4.22872826e-04]
 [ 8.90255978e-05]
 [-4.27667236e-04]
 [-1.55966259e-04]
 [-1.42089433e-04]
 [ 1.84490829e-04]
 [ 2.53113452e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 0.00010965]
 [ 0.00083291]
 [-0.00659711]
 [-0.01593772]
 [ 0.00335368]
 [-0.01611666]
 [-0.00587151]
 [-0.00535518]
 [ 0.00695075]
 [ 0.0095426 ]] 

Delta:
 [[ 3.05857178e-06]
 [ 2.21713660e-05]
 [-1.75435300e-04]
 [-4.23719666e-04]
 [ 8.92057781e-05]
 [-4.28526525e-04]
 [-1.56292831e-04]
 [-1.42374311e-04]
 [ 1.84863116e-04]
 [ 2.53615434e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 0.00011272]
 [ 0.00085512]
 [-0.0067729 ]
 [-0.01636229]
 [ 0.00344307]
 [-0.01654605]
 [-0.00602813]
 [-0.00549784]
 [ 0.00713599]
 [ 0.00979672]] 

Delta:
 [[ 3.0

Beta:
 [[ 0.00017029]
 [ 0.00126171]
 [-0.00999973]
 [-0.02415579]
 [ 0.00508413]
 [-0.02442848]
 [-0.00890601]
 [-0.00811657]
 [ 0.01053652]
 [ 0.01446046]] 

Delta:
 [[ 3.31261332e-06]
 [ 2.29108743e-05]
 [-1.82480255e-04]
 [-4.40748616e-04]
 [ 9.28171119e-05]
 [-4.45798211e-04]
 [-1.62914088e-04]
 [-1.48096189e-04]
 [ 1.92316618e-04]
 [ 2.63700752e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 0.00017362]
 [ 0.00128466]
 [-0.0101826 ]
 [-0.02459748]
 [ 0.00517715]
 [-0.02487524]
 [-0.00906929]
 [-0.00826498]
 [ 0.01072925]
 [ 0.01472472]] 

Delta:
 [[ 3.32615863e-06]
 [ 2.29468345e-05]
 [-1.82870577e-04]
 [-4.41696995e-04]
 [ 9.30175350e-05]
 [-4.46759422e-04]
 [-1.63285476e-04]
 [-1.48414200e-04]
 [ 1.92729552e-04]
 [ 2.64261864e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 0.00017696]
 [ 0.00130764]
 [-0.01036587]
 [-0.02504014]
 [ 0.00527037]
 [-0.02532296]
 [-0.00923295]
 [-0.00841371]
 [ 0.0109224 ]
 [ 0.01498955]] 

Delta:
 [[ 3.3

Beta:
 [[ 0.00023942]
 [ 0.00172694]
 [-0.01373405]
 [-0.0331775 ]
 [ 0.00698405]
 [-0.03355381]
 [-0.01224555]
 [-0.01114764]
 [ 0.01447246]
 [ 0.01985669]] 

Delta:
 [[ 3.58768865e-06]
 [ 2.35498816e-05]
 [-1.90681906e-04]
 [-4.60811555e-04]
 [ 9.70402107e-05]
 [-4.66106462e-04]
 [-1.70813346e-04]
 [-1.54797602e-04]
 [ 2.00990389e-04]
 [ 2.75556077e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 0.00024302]
 [ 0.00175052]
 [-0.01392517]
 [-0.03363937]
 [ 0.00708131]
 [-0.03402099]
 [-0.01241679]
 [-0.0113028 ]
 [ 0.01467391]
 [ 0.02013288]] 

Delta:
 [[ 3.60170929e-06]
 [ 2.35766082e-05]
 [-1.91114528e-04]
 [-4.61878525e-04]
 [ 9.72637846e-05]
 [-4.67184627e-04]
 [-1.71235449e-04]
 [-1.55152121e-04]
 [ 2.01447669e-04]
 [ 2.76185606e-04]] 

Mu:
 [0.055791301945490764, -0.07174378216208464] 

Beta:
 [[ 0.00024663]
 [ 0.00177412]
 [-0.01411672]
 [-0.03410233]
 [ 0.0071788 ]
 [-0.03448926]
 [-0.01258845]
 [-0.0114583 ]
 [ 0.01487582]
 [ 0.02040969]] 

Delta:
 [[ 3.6

KeyboardInterrupt: 

# Generated the data into files for R

In [981]:
np.savetxt('X1.csv', X1, delimiter=",")
np.savetxt('y1.csv', y1, delimiter=",")
np.savetxt('X2.csv', X2, delimiter=",")
np.savetxt('y2.csv', y2, delimiter=",")

# The followings are NOT true

## Step 1: Combine data to generate golden rules

In [564]:
col = []
for i in range(10): col += ['V'+str(i+1)]
X1 = pd.DataFrame(X1, columns=col)
X2 = pd.DataFrame(X2, columns=col)
# X = pd.concat([X1, X2], ignore_index = True)
y = np.concatenate((y1, y2))

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0})}{1 + \exp{(X_{ij}^\top\beta_0})}$$

In [566]:
def Pi(x, beta_0):
    return np.asarray((np.exp(x @ beta_0) / (1 + np.exp(x @ beta_0)))) #need to add /mu_{i0}?

Notation for $L_1$:

$$L_1 = \sum_{j=1}^{n_i}(y_{ij}X_{ij}-\pi_{ij}X_{ij})$$

In [567]:
def L_1(x, y, beta_0):
    return np.expand_dims(np.asarray((y * x - Pi(x, beta_0) * x).sum(axis = 0)), axis=1)

Notation for $L_2$:

$$L_2 = - \sum_{j=1}^{n_i}[\pi_{ij}(1-\pi_{ij})X_{ij}X_{ij}^\top]$$

In [568]:
def L_2(x, y, beta_0):
    XX = 0
    p = Pi(x, beta_0) * (1- Pi(x, beta_0))
    for i in range(x.shape[0]):
#         XX += p[i] * ((np.asarray(x.iloc[i,:]).reshape(x.shape[1],1)))**2 
        XX += p[i] * (np.asarray(x.iloc[i,:]).reshape(x.shape[1],1) \
        @ np.asarray(x.iloc[i,:]).reshape(x.shape[1],1).transpose())
    L2 = -XX
    return L2

In [588]:
beta_0 = np.repeat(0, 10).reshape(10,1)

In [1111]:
print('Initial beta:', beta_0, "\n")
for i in range(100):
    print('Step ', i+1, ':\n')
    beta_0
    beta_new = beta_0 - inv(L_2(X1, y1, beta_0) + L_2(X2, y2, beta_0)) \
    @ (L_1(X1, y1, beta_0) + L_1(X2, y2, beta_0))
    delta = beta_new - beta_0
    beta_0 = beta_new
    print('Beta:\n', beta_0, '\n')
    print('Delta:\n', delta, '\n')
    if (max(np.abs(delta)) == 0):
        break;

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :



TypeError: L_2() missing 2 required positional arguments: 'mu' and 'beta_0'

In [591]:
L_2(X1, y1, beta_0)+L_2(X2, y2, beta_0)

array([[-2017.92540591,   278.96089871,    55.13581563,    42.00608349,
           48.96279807,   210.41011784,   147.62714793,   352.95317426,
          260.13291691,    53.38371126],
       [  278.96089871, -1882.57363095,   104.82882103,   -71.34377803,
          332.49827995,   -40.34237896,   356.65541555,   265.13382507,
          523.47528537,   316.84156834],
       [   55.13581563,   104.82882103, -2018.44299488,   -17.02955605,
          -58.167005  ,    96.91290635,   -21.95912822,    81.27766099,
          252.23937134,   165.53816921],
       [   42.00608349,   -71.34377803,   -17.02955605, -1945.83029471,
           63.59659857,    38.07785122,    34.52403033,    49.45718014,
           84.64072536,    18.7808896 ],
       [   48.96279807,   332.49827995,   -58.167005  ,    63.59659857,
        -1959.36670765,    62.41459172,    57.82157479,   327.73823271,
          304.5362868 ,   118.92167227],
       [  210.41011784,   -40.34237896,    96.91290635,    38.07785122,
   

In [550]:
L_2(X1, y1, beta_0)

array([[-4140.35624389],
       [-4107.14077318],
       [-4096.38366046],
       [-4070.5526563 ],
       [-3959.86346214],
       [-4006.78389039],
       [-3930.36488249],
       [-4527.7504776 ],
       [-3984.77596394],
       [-4300.38682221]])

In [565]:
true_beta

array([[-0.0217251 ],
       [ 0.35104993],
       [-2.04337875],
       [-5.75784864],
       [ 1.19368298],
       [-5.78128915],
       [-2.31044591],
       [-2.13129101],
       [ 2.56380536],
       [ 3.40715245]])

In [587]:
inv(L_2(X1, y1, beta_0) + L_2(X2, y2, beta_0)) @ (L_1(X1, y1, beta_0) + L_1(X2, y2, beta_0))

array([[-0.10817893],
       [-0.15577158],
       [-0.04324669],
       [-0.0087236 ],
       [-0.11681485],
       [-0.05317466],
       [-0.07200324],
       [-0.17340093],
       [-0.20606506],
       [-0.11675683]])